In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.linear_model import Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df_train=pd.read_csv("C:\Users\91915\Desktop\Prodigy @ T-1\Datasets//train.csv")

In [ ]:
for col in df_train.columns:
    print(col)

In [ ]:
for col in df_train.columns:
    print(col)

In [ ]:

cat_var = df_train.select_dtypes(include = ["object"]).columns
num_var = df_train.select_dtypes(exclude = ["object"]).columns
num_var = num_var.drop("SalePrice")
print("Numerical variables : " + str(len(num_var)))
print("Categorical variables : " + str(len(cat_var)))


In [ ]:
df_train['SalePrice'].describe()

In [ ]:
df_train['SalePrice'].isna().sum()

In [ ]:
sns.displot(df_train['SalePrice'], kind='kde')

In [ ]:
print("Skewness: %f" % df_train['SalePrice'].skew())
print("Kurtosis: %f" % df_train['SalePrice'].kurt())

In [ ]:
missing =df_train.isnull().sum().sort_values(ascending=False)
total = df_train.isnull().count().sort_values(ascending=False)
NotNull =(total - missing).sort_values(ascending=True)
percent = ((df_train.isnull().sum()/df_train.isnull().count())*100).sort_values(ascending=False)
missing_data = pd.concat([NotNull, missing, total, percent], axis=1, keys=['NotNull ', 'Missing', 'total', 'Percent'])
missing_data.head(20)

In [ ]:
df0=missing_data.head(7)
col = df0.index
df1 = df_train.drop(col, axis=1)

In [ ]:
cat_var = df1.select_dtypes(exclude=np.number).columns
num_var = df1.select_dtypes(include=np.number).columns

df_num=df_train[num_var]
df_cat=df_train[cat_var]


In [ ]:
df_num = df_num.drop(['Id'], axis =1)

In [ ]:
#numerical variables
f = pd.melt(df_num)
g = sns.FacetGrid(f, col="variable",  col_wrap=3, sharex=False, sharey=False)
g = g.map(sns.distplot, "value")

In [ ]:
#pick variables by degree of correlation
corr = df_num.corr()
corr.sort_values(["SalePrice"], ascending = False, inplace = True)
print(corr.SalePrice)

In [ ]:
corrS= df_num.corr(method='spearman')
corrS.sort_values(['SalePrice'], ascending=False, inplace=True)
corrS.SalePrice.head(30)

In [ ]:
f = pd.melt(df_num, id_vars=['SalePrice'], value_vars=df_num.columns)
g = sns.FacetGrid(f, col="variable",  col_wrap=3, sharex=False, sharey=False)
g = g.map(sns.scatterplot, "value",  "SalePrice")

In [ ]:
#checking for missing values
df_num.isna().sum()

In [ ]:
#replace missing values with median 

df_num['GarageYrBlt']=df_num['GarageYrBlt'].fillna(df_num['GarageYrBlt'].median())
df_num['MasVnrArea']=df_num['MasVnrArea'].fillna(df_num['MasVnrArea'].median())

In [ ]:
df_num.shape

In [ ]:
fig, axes = plt.subplots(7, 2, figsize=(9, 16))
fig.suptitle('Boxplots for outlier detection')
sns.boxplot(ax=axes[0,0], x=df_num['SalePrice'])
sns.boxplot(ax=axes[0,1], x=df_num['OverallQual'])
sns.boxplot(ax=axes[1,0], x=df_num['GrLivArea'])
sns.boxplot(ax=axes[1,1], x=df_num['GarageCars'])
sns.boxplot(ax=axes[2,0], x=df_num['GarageArea'])
sns.boxplot(ax=axes[2,1], x=df_num['TotalBsmtSF'])
sns.boxplot(ax=axes[3,0], x=df_num['1stFlrSF'])
sns.boxplot(ax=axes[3,1], x=df_num['FullBath'])
sns.boxplot(ax=axes[4,0], x=df_num['TotRmsAbvGrd'])
sns.boxplot(ax=axes[4,1], x=df_num['YearBuilt'])
sns.boxplot(ax=axes[5,0], x=df_num['YearRemodAdd'])
sns.boxplot(ax=axes[5,1], x=df_num['GarageYrBlt'])
sns.boxplot(ax=axes[6,0], x=df_num['Fireplaces'])
plt.show();

In [ ]:
def outlier_imputer(column_list, iqr_factor):
    
    for col in column_list:
        # Reassign minimum to zero
        df_num.loc[df_num[col] < 0, col] = 0

        # Calculate upper threshold
        q1 = df_num[col].quantile(0.25)
        q3 = df_num[col].quantile(0.75)
        iqr = q3 - q1
        upper_threshold = q3 + (iqr_factor * iqr)
        print(col)
        print('q3:', q3)
        print('upper_threshold:', upper_threshold)

        # Reassign values > threshold to threshold
        df_num.loc[df_num[col] > upper_threshold, col] = upper_threshold
        print(df_num[col].describe())
        print()

In [ ]:
outlier_imputer(['SalePrice'], 4.5)

In [ ]:
df_num.skew(axis=0)

In [ ]:
skewness = df_num.skew(axis=0)
skewness = skewness[abs(skewness) > 0.5 ]
print(str(skewness.shape[0]) + " skewed numerical features to log transform")
skewed_features = skewness.index
df_num[skewed_features] = np.log1p(df_num[skewed_features])

In [ ]:
df_cat.shape

In [ ]:
df_cat = pd.concat([df_cat, df_num['SalePrice']], axis=1)

In [ ]:
def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)
f = pd.melt(df_cat, id_vars=['SalePrice'], value_vars=df_cat)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False)
g = g.map(boxplot, "value", "SalePrice")

In [ ]:
from scipy.stats import f_oneway
def ANOVA_Test(df, feature):
    category_groups = df.groupby(feature)['SalePrice'].apply(list)
    return f_oneway(*category_groups)

In [ ]:
p_vals = {}
for col in df_cat.columns:
    p_vals[col] = ANOVA_Test(df_cat, col).statistic
p_vals.pop('SalePrice')

In [ ]:
sorted(p_vals, key=p_vals.get, reverse=True)[:10]

In [ ]:
df_cat=df_cat.drop('SalePrice', axis=1)

In [ ]:
purge1=corrS.SalePrice.head(20).index

purge2=sorted(p_vals, key=p_vals.get, reverse=True)[:10]

df2_cat= df_cat[purge2]
df2_num= df_num[purge1]

In [ ]:
df = pd.concat([df2_cat, df2_num], axis=1)

In [ ]:
df.shape

In [ ]:
df2 = df.copy()

In [ ]:
df2 = df.copy()

In [ ]:
y = df2['SalePrice']

X = df2.drop(['SalePrice'], axis=1)

In [ ]:
#instantiate scaler
scaler = StandardScaler()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
lss=Lasso()

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=0)

parameters = {'alpha': (0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1, 10)}

grid_search = GridSearchCV(lss, parameters, n_jobs=-1, scoring='r2', cv=cv)

grid_search = grid_search.fit(X_train, y_train)
grid_search.best_estimator_

In [ ]:
y_pred_train = grid_search.best_estimator_.predict(X_train)
print('r2: %.3f' % grid_search.best_score_)
print('MSE:', mean_squared_error(y_train, y_pred_train))
print('RMSE:',np.sqrt(mean_squared_error(y_train, y_pred_train)))
print('Config: %s' % grid_search.best_params_)

In [ ]:
#try on test subset
r_sq_test = grid_search.best_estimator_.score(X_test, y_test)
print('Coefficient of determination:', r_sq_test)
y_pred_test = grid_search.best_estimator_.predict(X_test)
print('R^2:', r2_score(y_test, y_pred_test))
print('MSE:', mean_squared_error(y_test, y_pred_test))
print('RMSE:',np.sqrt(mean_squared_error(y_test, y_pred_test)))

In [ ]:
table =({'model': 'Lr Reg',
                          'R2':0.84,
                          'RMSE': 0.16
                          },
                         )

champion =pd.DataFrame(table)
champion

In [ ]:
poly = PolynomialFeatures(degree=2)
X_poly=  poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=0)

In [ ]:
cv1 = RepeatedKFold(n_splits=10, n_repeats=5, random_state=0)
grid_search = GridSearchCV(lss, parameters, n_jobs=-1, scoring='r2', cv=cv1)

grid_search = grid_search.fit(X_train, y_train)
grid_search.best_estimator_

In [ ]:
y_pred_train = grid_search.best_estimator_.predict(X_train)

In [ ]:
print('r2: %.3f' % grid_search.best_score_)
print('MSE:', mean_squared_error(y_train, y_pred_train))
print('RMSE:',np.sqrt(mean_squared_error(y_train, y_pred_train)))
print('Config: %s' % grid_search.best_params_)

In [ ]:
r_sq_test = grid_search.best_estimator_.score(X_test, y_test)
print('Coefficient of determination:', r_sq_test)
y_pred_test = grid_search.best_estimator_.predict(X_test)
print('R^2:', r2_score(y_test, y_pred_test))
print('MSE:', mean_squared_error(y_test, y_pred_test))
print('RMSE:',np.sqrt(mean_squared_error(y_test, y_pred_test)))

In [ ]:
poly_table =({'model': 'Pl Reg',
                          'R2':0.86,
                          'RMSE': 0.14
                          },
                         )
poly_result = pd.DataFrame(poly_table)

champion =  pd.concat([champion, poly_result], axis=0)

In [ ]:
del df_train
df_train=pd.read_csv("C:\Users\91915\Desktop\Tasks\PRODIGY_ML_01\PRODIGY_ML_01\Datasets//train.csv")

In [ ]:
# PoolQC : data description says NA means no pool
df_train.loc[:, "PoolQC"] = df_train.loc[:, "PoolQC"].fillna("None")
# MiscFeature : NA most likely means no noteworthy feature
df_train.loc[:, "MiscFeature"] = df_train.loc[:, "MiscFeature"].fillna("None")
# Alley : NA most likely means no Alley
df_train.loc[:, "Alley"] = df_train.loc[:, "Alley"].fillna("None")
# No fence
df_train.loc[:, "Fence"] = df_train.loc[:, "Fence"].fillna("None")
# No veneer on masonry
df_train.loc[:, "MasVnrType"] = df_train.loc[:, "MasVnrType"].fillna("None")
# No fire place
df_train.loc[:, "FireplaceQu"] = df_train.loc[:, "FireplaceQu"].fillna("No")
# No frontage ft or m
df_train.loc[:, "LotFrontage"] = df_train.loc[:, "LotFrontage"].fillna(0)
# GarageType etc : data description says NA for garage features is "no garage"
df_train.loc[:, "GarageType"] = df_train.loc[:, "GarageType"].fillna("No")
df_train.loc[:, "GarageFinish"] = df_train.loc[:, "GarageFinish"].fillna("No")
df_train.loc[:, "GarageQual"] = df_train.loc[:, "GarageQual"].fillna("No")
df_train.loc[:, "GarageCond"] = df_train.loc[:, "GarageCond"].fillna("No")
df_train.loc[:, "GarageArea"] = df_train.loc[:, "GarageArea"].fillna(0)
df_train.loc[:, "GarageCars"] = df_train.loc[:, "GarageCars"].fillna(0)
df_train.loc[:, "GarageYrBlt"] = df_train.loc[:, "GarageYrBlt"].fillna(0)

# BsmtQual etc : data description says NA for basement features is "no basement"
df_train.loc[:, "BsmtQual"] = df_train.loc[:, "BsmtQual"].fillna("No")
df_train.loc[:, "BsmtCond"] = df_train.loc[:, "BsmtCond"].fillna("No")
df_train.loc[:, "BsmtExposure"] = df_train.loc[:, "BsmtExposure"].fillna("No")
df_train.loc[:, "BsmtFinType1"] = df_train.loc[:, "BsmtFinType1"].fillna("No")
df_train.loc[:, "BsmtFinType2"] = df_train.loc[:, "BsmtFinType2"].fillna("No")
# No veneer area
df_train.loc[:, "MasVnrArea"] = df_train.loc[:, "MasVnrArea"].fillna(0)
#misc feature value
df_train.loc[:, "MiscVal"] = df_train.loc[:, "MiscVal"].fillna(0)


# for electrical, there is no way to tell if it means that 
#there are no electrical works in the building, but the 
#utilities means that it possesses allpub, we will make it the mode later on 

In [ ]:
df_train['PoolQC_B'] = np.where(df_train['PoolQC']=='None', 0, 1)

df_train['MiscFeature_B'] = np.where(df_train['MiscFeature']=='None', 0, 1)

df_train['Alley_B'] = np.where(df_train['Alley']=='None', 0, 1)

df_train['Fence_B'] = np.where(df_train['Fence']=='None', 0, 1)

df_train['MasVnrType_B'] = np.where(df_train['MasVnrType']=='None', 0, 1)

df_train['GarageQual_B'] = np.where(df_train['MasVnrType']=='No', 0, 1)

df_train['BsmtQual_B'] = np.where(df_train['MasVnrType']=='No', 0, 1)

In [ ]:
df_train=df_train.replace({"MSZoning" : {"FV" : 4, "RL" : 3, 'RH': 2, 'RM':2, 'C(all)':1},
                       "LotConfig" : {"CulDSac" : 1, "FR3" : 1, "FR2" : 2, "Corner" : 2, "Inside" : 2},
                       "Neighborhood" : {"NridgHt" : 3, "NoRidge" : 3, "StoneBr": 3, "Timber" : 2,  "Somerst" : 2, "Veenker": 2, "Crawfor" : 2, "ClearCr" : 2, "CollgCr": 2, 
                                         "Blmngtn" : 2, "NWAmes" : 2, "Gilbert": 2, "SawyerW" : 2, "Mitchel" : 1, "NPkVill": 1, 
                                         "NAmes" : 1, "SWISU" : 1, "Blueste": 1, "Sawyer" : 1, "BrkSide" : 1, "Edwards": 1, "OldTown" : 1,
                                         "BrDale" : 1, "IDOTRR": 1, "MeadowV" : 1},
                       "Condition2" : {"PosA" : 3, "PosN" : 3, "RRAe": 2, "Norm" : 2, "RRAn" : 1, 
                                         "Feedr" : 1, "Artery" : 1, "RRNn" : 1},
                       "BsmtFinType2" : {"Unf" : 2, "LwQ": 2, "Rec" : 1, "BLQ" : 1, 
                                         "ALQ" : 2, "GLQ" : 3},
                       "BldgType" : {"TwnhsE" : 2, "1Fam" : 2, "Twnhs" : 1, "Duplex": 1, "2fmCon" : 1},
                       "Foundation" : {"PConc" : 3, "Wood" : 2, "CBlock": 2, "Stone": 2, "BrkTil": 2, "Slab" : 1},
                       "HeatingQC" : {"Po" : 1, "Fa" : 2, "TA": 2, "Gd": 2, "Ex" : 3},
                       "SaleType" : {"Con" : 0, "New" : 1, "CWD" : 2, "WD": 3, "ConLw" : 4, "ConLD" : 5 ,
                                     "ConLI": 3, "COD" : 4, "Oth" : 5},
                       "SaleCondition" : {"Partial" : 1, "Normal" : 2, "Alloca": 3, "Family": 4, "Abnorml" : 5 , "AdjLand" : 5},
                       "CentralAir" : {"Y" : 2, "N" : 1},
                       "FireplaceQu" : {"No" : 1,"Po" : 1, "Fa" : 1, "TA" : 1, "Gd" : 1, "Ex" : 2},
                       "MasVnrType" : {"Stone" : 3, "BrkFace" : 2, "BrkCmn" : 1, "None" : 1},
                       "MiscFeature" : {"TenC" : 3, "Gar2" : 2, "None" : 2, "Shed" : 2, "Othr": 1},
                       "Electrical" : {"SBrkr" : 1, "FuseA" : 2, "FuseF" : 3, "FuseP" : 4, "Mixed": 5}}
                     )

In [ ]:
df_train = df_train.replace({"BsmtCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "BsmtQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
                       "ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "GarageCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "GarageQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}}
                     )

In [ ]:
df_train["OverallScore"] = df_train["OverallQual"] * df_train["OverallCond"]

df_train["ExterScore"] = df_train["ExterQual"] * df_train["ExterCond"]

df_train["BsmtScore"] = df_train["BsmtQual"] * df_train["BsmtCond"]

df_train["FlrSF"] = df_train["1stFlrSF"] + df_train["2ndFlrSF"]

df_train["GarageScore"] = df_train["GarageQual"] * df_train["GarageCond"]

df_train["LivSF"] = df_train["GrLivArea"] + df_train["TotalBsmtSF"]

df_train["Exteroverall"] = df_train["ExterQual"] * df_train["OverallCond"]

df_train["AddedScore"] = df_train["BsmtQual"] * df_train["GarageQual"]

In [ ]:
# binning continuous variables into buckets based on quantiles
df_train['AgeBldg'] = pd.qcut(df_train['YearBuilt'], q=[0, .2, .4, .6, .8, 1], 
                             labels=['Ancient', 'Vintage', 'Old', 'Average', 'New'])

df_train['LastRemod']= 2010 - df_train['YearRemodAdd']

df_train['StateRemod'] = pd.qcut(df_train['LastRemod'], q=[0, .2, .4, .6, .8, 1], 
                             labels=['Critical', 'Due', 'Partial', 'Optimal', 'Newly Remod'])

df_train['LotFrontageRn'] = pd.qcut(df_train['LotFrontage'], q=[0, .2, .4, .6, .8, 1], 
                             labels=['Little', 'Fair', 'Adequate', 'Good', 'Excellent'])

df_train['LotAreaRn'] = pd.qcut(df_train['LotArea'], q=[0, .2, .4, .6, .8, 1], 
                             labels=['Little', 'Fair', 'Adequate', 'Good', 'Excellent'])


In [ ]:
df_train['MasVnrCov']= pd.cut(df_train['MasVnrArea'], bins=[-1, 400, 800, 1200, 1601], 
                             labels=['Small', 'Sizeable', 'Optimal', 'Excellent'])

df_train['MiscValRn'] = pd.cut(df_train['MiscVal'], bins=[-1, 100, 1000, 5000, 15501], 
                             labels=['Not Valuable', 'Fair', 'Good', 'Great'])



In [ ]:
cat_var = df_train.select_dtypes(exclude=np.number).columns
num_var = df_train.select_dtypes(include=np.number).columns

df_num=df_train[num_var]
df_cat=df_train[cat_var]

In [ ]:
df_num.shape

In [ ]:
outlier_imputer(['SalePrice'], 4)

In [ ]:
corrS2= df_num.corr(method='spearman')
corrS2.sort_values(['SalePrice'], ascending=False, inplace=True)
corrS2.SalePrice.head(25)

In [ ]:
corrS= df_num.corr(method='pearson')
corrS.sort_values(['SalePrice'], ascending=False, inplace=True)
corrS.SalePrice.head(24)

In [ ]:
df_cat = pd.concat([df_cat, df_num['SalePrice']], axis=1)

In [ ]:
from scipy.stats import f_oneway
def ANOVA_Test(df, feature):
    category_groups = df.groupby(feature)['SalePrice'].apply(list)
    return f_oneway(*category_groups)

In [ ]:
p_vals = {}
for col in df_cat.columns:
    p_vals[col] = ANOVA_Test(df_cat, col).statistic
p_vals.pop('SalePrice')

In [ ]:
sorted(p_vals, key=p_vals.get, reverse=True)[:10]

In [ ]:
purge1=corrS2.SalePrice.head(7).index

purge2=sorted(p_vals, key=p_vals.get, reverse=True)[:6]

df2_cat= df_cat[purge2]
df2_num= df_num[purge1]


In [ ]:
skewness = df2_num.skew(axis=0)
skewness = skewness[abs(skewness) > 0.5 ]
print(str(skewness.shape[0]) + " skewed numerical features to log transform")
skewed_features = skewness.index
df2_num[skewed_features] = np.log1p(df2_num[skewed_features])

In [ ]:
df_full = pd.concat([df2_cat, df2_num], axis=1)

In [ ]:
df_full.isna().sum()

In [ ]:
df1 = df_full.copy()

In [ ]:
df1=pd.get_dummies(df1, drop_first=False)

In [ ]:
y = df1['SalePrice']

X = df1.drop(['SalePrice'], axis=1)

In [ ]:
scaler = StandardScaler()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
lss=Lasso()

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=0)

parameters = {'alpha': (0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1, 10)}

grid_search = GridSearchCV(lss, parameters, n_jobs=-1, scoring='r2', cv=cv)

grid_search = grid_search.fit(X_train, y_train)
grid_search.best_estimator_

In [ ]:
y_pred_train = grid_search.best_estimator_.predict(X_train)
print('r2: %.3f' % grid_search.best_score_)
print('MSE:', mean_squared_error(y_train, y_pred_train))
print('RMSE:',np.sqrt(mean_squared_error(y_train, y_pred_train)))
print('Config: %s' % grid_search.best_params_)

In [ ]:
r_sq_test = grid_search.best_estimator_.score(X_test, y_test)
print('Coefficient of determination:', r_sq_test)
y_pred_test = grid_search.best_estimator_.predict(X_test)
print('R^2:', r2_score(y_test, y_pred_test))
print('MSE:', mean_squared_error(y_test, y_pred_test))
print('RMSE:',np.sqrt(mean_squared_error(y_test, y_pred_test)))

In [ ]:
poly = PolynomialFeatures(degree=2)
X_poly=  poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=0)



In [ ]:
cv1 = RepeatedKFold(n_splits=10, n_repeats=5, random_state=0)
grid_search = GridSearchCV(lss, parameters, n_jobs=-1, scoring='r2', cv=cv1)

grid_search = grid_search.fit(X_train, y_train)
grid_search.best_estimator_

In [ ]:
y_pred_train = grid_search.best_estimator_.predict(X_train)

In [ ]:
print('r2: %.3f' % grid_search.best_score_)
print('MSE:', mean_squared_error(y_train, y_pred_train))
print('RMSE:',np.sqrt(mean_squared_error(y_train, y_pred_train)))
print('Config: %s' % grid_search.best_params_)

In [ ]:
r_sq_test = grid_search.best_estimator_.score(X_test, y_test)
print('Coefficient of determination:', r_sq_test)
y_pred_test = grid_search.best_estimator_.predict(X_test)
print('R^2:', r2_score(y_test, y_pred_test))
print('MSE:', mean_squared_error(y_test, y_pred_test))
print('RMSE:',np.sqrt(mean_squared_error(y_test, y_pred_test)))

In [ ]:
table2 =({'model': 'FE_LR Reg',
                          'R2':0.83,
                          'RMSE': 0.16
                          },
             {'model': 'FE_Poly Reg',
                          'R2':0.83,
                          'RMSE': 0.16
                          },
                         )
result22 = pd.DataFrame(table2)

champion =  pd.concat([champion, result22], axis=0)
champion

In [ ]:
results = pd.DataFrame(data={'actual': (np.expm1(y_test)),
                             'predicted': (np.expm1(y_pred_test.ravel()))})
results['residual'] = results['actual'] - results['predicted']
results.head()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
sns.set(style='whitegrid')
sns.scatterplot(x='actual',
                y='predicted',
                data=results,
                s=20,
                alpha=0.5,
                ax=ax
)
plt.plot([0,600000], [0,600000], c='red', linewidth=2)
plt.title('Actual vs. predicted');

In [ ]:
sns.histplot(results['residual']) #bins=np.arange(-15,15.5,0.5))
plt.title('Distribution of the residuals')
plt.xlabel('residual value')
plt.ylabel('count');

In [ ]:
sns.scatterplot(x='predicted', y='residual', data=results)
plt.axhline(0, c='red')
plt.title('Scatterplot of residuals over predicted values')
plt.xlabel('predicted value')
plt.ylabel('residual value')
plt.show()

NameError: name 'sns' is not defined

In [ ]:
df_test=pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

In [ ]:
len(df_test['PoolQC'])

In [ ]:
# PoolQC : data description says NA means no pool
df_test.loc[:, "PoolQC"] = df_test.loc[:, "PoolQC"].fillna("None")
# MiscFeature : NA most likely means no noteworthy feature
df_test.loc[:, "MiscFeature"] = df_test.loc[:, "MiscFeature"].fillna("None")
# Alley : NA most likely means no Alley
df_test.loc[:, "Alley"] = df_test.loc[:, "Alley"].fillna("None")
# No fence
df_test.loc[:, "Fence"] = df_test.loc[:, "Fence"].fillna("None")
# No veneer on masonry
df_test.loc[:, "MasVnrType"] = df_test.loc[:, "MasVnrType"].fillna("None")
# No fire place
df_test.loc[:, "FireplaceQu"] = df_test.loc[:, "FireplaceQu"].fillna("No")
# No frontage ft or m
df_test.loc[:, "LotFrontage"] = df_test.loc[:, "LotFrontage"].fillna(0)
# GarageType etc : data description says NA for garage features is "no garage"
df_test.loc[:, "GarageType"] = df_test.loc[:, "GarageType"].fillna("No")
df_test.loc[:, "GarageFinish"] = df_test.loc[:, "GarageFinish"].fillna("No")
df_test.loc[:, "GarageQual"] = df_test.loc[:, "GarageQual"].fillna("No")
df_test.loc[:, "GarageCond"] = df_test.loc[:, "GarageCond"].fillna("No")
df_test.loc[:, "GarageArea"] = df_test.loc[:, "GarageArea"].fillna(0)
df_test.loc[:, "GarageCars"] = df_test.loc[:, "GarageCars"].fillna(0)
df_test.loc[:, "GarageYrBlt"] = df_test.loc[:, "GarageYrBlt"].fillna(0)

# BsmtQual etc : data description says NA for basement features is "no basement"
df_test.loc[:, "BsmtQual"] = df_test.loc[:, "BsmtQual"].fillna("No")
df_test.loc[:, "BsmtCond"] = df_test.loc[:, "BsmtCond"].fillna("No")
df_test.loc[:, "BsmtExposure"] = df_test.loc[:, "BsmtExposure"].fillna("No")
df_test.loc[:, "BsmtFinType1"] = df_test.loc[:, "BsmtFinType1"].fillna("No")
df_test.loc[:, "BsmtFinType2"] = df_test.loc[:, "BsmtFinType2"].fillna("No")
# No veneer area
df_test.loc[:, "MasVnrArea"] = df_test.loc[:, "MasVnrArea"].fillna(0)
#misc feature value
df_test.loc[:, "MiscVal"] = df_test.loc[:, "MiscVal"].fillna(0)
#kitchen quality
df_test.loc[:, 'KitchenQual'] = df_test.loc[:, 'KitchenQual'].fillna("Fa")


In [ ]:
df_test['PoolQC_B'] = np.where(df_test['PoolQC']=='None', 0, 1)

df_test['MiscFeature_B'] = np.where(df_test['MiscFeature']=='None', 0, 1)

df_test['Alley_B'] = np.where(df_test['Alley']=='None', 0, 1)

df_test['Fence_B'] = np.where(df_test['Fence']=='None', 0, 1)

df_test['MasVnrType_B'] = np.where(df_test['MasVnrType']=='None', 0, 1)

df_test['GarageQual_B'] = np.where(df_test['MasVnrType']=='No', 0, 1)

df_test['BsmtQual_B'] = np.where(df_test['MasVnrType']=='No', 0, 1)

In [ ]:
df_test=df_test.replace({"MSZoning" : {"FV" : 4, "RL" : 3, 'RH': 2, 'RM':2, 'C(all)':1},
                       "LotConfig" : {"CulDSac" : 1, "FR3" : 1, "FR2" : 2, "Corner" : 2, "Inside" : 2},
                       "Neighborhood" : {"NridgHt" : 3, "NoRidge" : 3, "StoneBr": 3, "Timber" : 2,  "Somerst" : 2, "Veenker": 2, "Crawfor" : 2, "ClearCr" : 2, "CollgCr": 2, 
                                         "Blmngtn" : 2, "NWAmes" : 2, "Gilbert": 2, "SawyerW" : 2, "Mitchel" : 1, "NPkVill": 1, 
                                         "NAmes" : 1, "SWISU" : 1, "Blueste": 1, "Sawyer" : 1, "BrkSide" : 1, "Edwards": 1, "OldTown" : 1,
                                         "BrDale" : 1, "IDOTRR": 1, "MeadowV" : 1},
                       "Condition2" : {"PosA" : 3, "PosN" : 3, "RRAe": 2, "Norm" : 2, "RRAn" : 1, 
                                         "Feedr" : 1, "Artery" : 1, "RRNn" : 1},
                       "BsmtFinType2" : {"Unf" : 2, "LwQ": 2, "Rec" : 1, "BLQ" : 1, 
                                         "ALQ" : 2, "GLQ" : 3},
                       "BldgType" : {"TwnhsE" : 2, "1Fam" : 2, "Twnhs" : 1, "Duplex": 1, "2fmCon" : 1},
                       "Foundation" : {"PConc" : 3, "Wood" : 2, "CBlock": 2, "Stone": 2, "BrkTil": 2, "Slab" : 1},
                       "HeatingQC" : {"Po" : 1, "Fa" : 2, "TA": 2, "Gd": 2, "Ex" : 3},
                       "SaleType" : {"Con" : 0, "New" : 1, "CWD" : 2, "WD": 3, "ConLw" : 4, "ConLD" : 5 ,
                                     "ConLI": 3, "COD" : 4, "Oth" : 5},
                       "SaleCondition" : {"Partial" : 1, "Normal" : 2, "Alloca": 3, "Family": 4, "Abnorml" : 5 , "AdjLand" : 5},
                       "CentralAir" : {"Y" : 2, "N" : 1},
                       "FireplaceQu" : {"No" : 1,"Po" : 1, "Fa" : 1, "TA" : 1, "Gd" : 1, "Ex" : 2},
                       "MasVnrType" : {"Stone" : 3, "BrkFace" : 2, "BrkCmn" : 1, "None" : 1},
                       "MiscFeature" : {"TenC" : 3, "Gar2" : 2, "None" : 2, "Shed" : 2, "Othr": 1},
                       "Electrical" : {"SBrkr" : 1, "FuseA" : 2, "FuseF" : 3, "FuseP" : 4, "Mix": 5}}
                     )

In [ ]:
df_test = df_test.replace({"BsmtCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "BsmtQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
                       "ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "GarageCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "GarageQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}}
                     )

In [ ]:
df_test["OverallScore"] = df_test["OverallQual"] * df_test["OverallCond"]

df_test["ExterScore"] = df_test["ExterQual"] * df_test["ExterCond"]

df_test["BsmtScore"] = df_test["BsmtQual"] * df_test["BsmtCond"]

df_test["FlrSF"] = df_test["1stFlrSF"] + df_test["2ndFlrSF"]

df_test["GarageScore"] = df_test["GarageQual"] * df_test["GarageCond"]

df_test["LivSF"] = df_test["GrLivArea"] + df_test["TotalBsmtSF"]

df_test["Exteroverall"] = df_test["ExterQual"] * df_test["OverallCond"]

df_test["AddedScore"] = df_test["BsmtQual"] * df_test["GarageQual"]

In [ ]:
# binning continuous variables into buckets based on quantiles
df_test['AgeBldg'] = pd.qcut(df_test['YearBuilt'], q=[0, .2, .4, .6, .8, 1], 
                             labels=['Ancient', 'Vintage', 'Old', 'Average', 'New'])

df_test['LastRemod']= 2010 - df_test['YearRemodAdd']

df_test['StateRemod'] = pd.qcut(df_test['LastRemod'], q=[0, .2, .4, .6, .8, 1], 
                             labels=['Critical', 'Due', 'Partial', 'Optimal', 'Newly Remod'])

df_test['LotFrontageRn'] = pd.qcut(df_test['LotFrontage'], q=[0, .2, .4, .6, .8, 1], 
                             labels=['Little', 'Fair', 'Adequate', 'Good', 'Excellent'])

df_test['LotAreaRn'] = pd.qcut(df_test['LotArea'], q=[0, .2, .4, .6, .8, 1], 
                             labels=['Little', 'Fair', 'Adequate', 'Good', 'Excellent'])

In [ ]:
df_test['MasVnrCov']= pd.cut(df_test['MasVnrArea'], bins=[-1, 400, 800, 1200, 1601], 
                             labels=['Small', 'Sizeable', 'Optimal', 'Excellent'])

df_test['MiscValRn'] = pd.cut(df_test['MiscVal'], bins=[-1, 100, 1000, 5000, 15501], 
                             labels=['Not Valuable', 'Fair', 'Good', 'Great'])


In [ ]:
cat_var = df_test.select_dtypes(exclude=np.number).columns
num_var = df_test.select_dtypes(include=np.number).columns

df_num=df_test[num_var]
df_cat=df_test[cat_var]

In [ ]:
df_num.isna().sum()

In [ ]:
purge3 = ['LivSF', 'OverallQual', 'FlrSF', 'Neighborhood',
       'GrLivArea', 'AddedScore']

In [ ]:

df2_cat= df_cat[purge2]
df2_num= df_num[purge3]

In [ ]:
skewness = df2_num.skew(axis=0)
skewness = skewness[abs(skewness) > 0.5 ]
print(str(skewness.shape[0]) + " skewed numerical features to log transform")
skewed_features = skewness.index
df2_num[skewed_features] = np.log1p(df2_num[skewed_features])

In [ ]:
df_full = pd.concat([df2_cat, df2_num], axis=1)

In [ ]:
df_full[['LivSF']] = df_full[['LivSF']].fillna(0)

In [ ]:
df_full.isna().sum()

In [ ]:
df = df_full.copy()

In [ ]:
df=pd.get_dummies(df, drop_first=False)

In [ ]:
X_poly1=  poly.transForm(X_work)

In [ ]:
SalePrice = grid_search.best_estimator_.predict(X_poly1)

In [ ]:

SalePrice = np.expm1(SalePrice)
SalePrice

In [ ]:
submission = pd.DataFrame({'Id': df_test['Id'], 'SalePrice': SalePrice})

# If the 'Id' column is your index, you can reset it as follows:
submission.reset_index(drop=True, inplace=True)
submission.to_csv('sample_submission.csv',index = False)
submission